In [0]:
import os
os.environ['KAGGLE_USERNAME'] = ""
os.environ['KAGGLE_KEY'] = ""
!kaggle competitions download -c competitive-data-science-final-project

In [0]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split

items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
test = pd.read_csv('test.csv.gz')
train = pd.read_csv('sales_train.csv.gz')
sample_submission = pd.read_csv('sample_submission.csv.gz')

# convert train dataset string date to date time
train.date = pd.to_datetime(train.date,format='%d.%m.%Y')

In [47]:
# joined data frame train

train_joined = (
    train
    .join(shops.set_index('shop_id'),on='shop_id')
    .join(items.set_index('item_id'),on='item_id')
    .join(item_categories.set_index('item_category_id'),on='item_category_id')  
)
train_joined.head(1)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
0,2013-01-02,0,59,22154,999.0,1.0,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray


,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
5,5,0.5
6,6,0.5
7,7,0.5
8,8,0.5
9,9,0.5
